In [2]:
!git clone https://github.com/piEsposito/br-quad-2.0

Cloning into 'br-quad-2.0'...
remote: Enumerating objects: 27, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 27 (delta 8), reused 20 (delta 4), pack-reused 0
Receiving objects: 100% (27/27), 11.56 MiB | 8.49 MiB/s, done.
Resolving deltas: 100% (8/8), done.


In [4]:
import json

with open('/content/br-quad-2.0/data/brquad-gte-dev-v2.0.json') as f:
  data = json.load(f)["data"]

with open('/content/br-quad-2.0/data/brquad-gte-dev-v2.0-data.json', "w") as f:
        json.dump(data, f)


In [5]:
len(data)

35

In [7]:
examples = []
for example in data:
    print()
    title = example['title']
    paragraph = example["paragraphs"][0]
    possible_qas = [qa for qa in example["qas"] if not qa["is_impossible"]]

    break

{'context': 'Os normandos (normandos: nourmands; francês: normandos; latim: normanni) eram as pessoas que nos séculos 10 e 11 deram seu nome à Normandia, uma região da França. Eles eram descendentes de nórdicos ("normandos" vem de "nórdicos") e piratas da Dinamarca, Islândia e Noruega que, sob seu líder Rollo, concordaram em jurar lealdade ao rei Carlos III de Francia Ocidental. Através de gerações de assimilação e mistura com as populações nativas francas e gaulesas romanas, seus descendentes gradualmente se fundiram com as culturas carolíngia da Francia Ocidental. A identidade cultural e étnica distinta dos normandos surgiu inicialmente na primeira metade do século 10 e continuou a evoluir nos séculos seguintes.', 'qas': [{'id': '56ddde6b9a695914005b9628', 'question': 'Em que país a Normandia está localizada?', 'is_impossible': False, 'answers': [{'answer_start': 157, 'text': 'França'}, {'answer_start': 157, 'text': 'França'}, {'answer_start': 157, 'text': 'França'}, {'answer_start':

In [ ]:
data

In [ ]:
import datasets as dts


dataset = dts.load_dataset('piEsposito/br-quad-2.0')
dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1429: FutureWarning: The repository for piEsposito/br-quad-2.0 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/piEsposito/br-quad-2.0
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `dataset

ModuleNotFoundError: No module named 'datasets_modules.datasets.piEsposito--br-quad-2'